## 1. Data Cleaning

In [3]:
import numpy as np 
import pandas as pd
import os
from tabulate import tabulate
import time
import pickle
import textwrap

import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
style.use('seaborn-v0_8-darkgrid')
plt.rcParams['font.family'] = 'monospace'

%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder, PolynomialFeatures
from sklearn.compose import make_column_transformer
from sklearn.inspection import permutation_importance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, make_scorer, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import make_pipeline
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform, euclidean

RANDOM_STATE = 42

In [4]:
df_movie = pd.read_csv('../data/raw/movies_details.csv')
df_movie.head(5)

,title,img_url,year,director,stars,duration,genres,overview,rating,num_rating,num_user_review,num_critic_review,budget,gross,country,metascore,oscar,win_and_nomination,url
0,Woman of the Hour,https://m.media-amazon.com/images/M/MV5BYzliNz...,2023,Anna Kendrick,"Anna Kendrick,Daniel Zovatto,Tony Hale",1h 35m,"Crime,Drama,Mystery","Sheryl Bradshaw, a single woman looking for a ...",6.7,24K,193,102,NaN,"$817,151",United States,74.0,NaN,1 win,https://www.imdb.com/title/tt7737800/?ref_=sr_t_1
1,The Substance,https://m.media-amazon.com/images/M/MV5BZDQ1NG...,2024,Coralie Fargeat,"Demi Moore,Margaret Qualley,Dennis Quaid",2h 21m,"Drama,Horror",A fading celebrity takes a black-market drug: ...,7.6,86K,883,279,"$17,500,000 (estimated)","$41,691,175",United Kingdom,78.0,NaN,3 wins & 6 nominations,https://www.imdb.com/title/tt17526714/?ref_=sr...
2,Venom: The Last Dance,https://m.media-amazon.com/images/M/MV5BZDMyYW...,2024,Kelly Marcel,"Tom Hardy,Chiwetel Ejiofor,Juno Temple",1h 49m,"Action,Adventure,Sci-Fi,Thriller","Eddie and Venom, on the run, face pursuit from...",6.2,21K,215,112,"$120,000,000 (estimated)","$182,673,243",United States,41.0,NaN,NaN,https://www.imdb.com/title/tt16366836/?ref_=sr...
3,Terrifier 3,https://m.media-amazon.com/images/M/MV5BNzc2MW...,2024,Damien Leone,"Lauren LaVera,David Howard Thornton,Antonella ...",2h 5m,Horror,Art the Clown is set to unleash chaos on the u...,6.7,19K,306,106,"$2,000,000 (estimated)","$57,652,158",United States,61.0,NaN,NaN,https://www.imdb.com/title/tt27911000/?ref_=sr...
4,Smile 2,https://m.media-amazon.com/images/M/MV5BYTg5OT...,2024,Parker Finn,"Naomi Scott,Rosemarie DeWitt,Lukas Gage",2h 7m,"Horror,Mystery,Thriller","About to embark on a world tour, global pop se...",7.2,23K,342,159,"$28,000,000 (estimated)","$86,109,996",United States,66.0,NaN,NaN,https://www.imdb.com/title/tt29268110/?ref_=sr...


In [5]:
df_rating = pd.read_csv('../data/raw/movie_user_rating.csv')
df_rating.head(5)

,user_id,movie_id,rating
0,ur127508339,tt7737800,8
1,ur6312156,tt7737800,8
2,ur150453978,tt7737800,1
3,ur18519281,tt7737800,3
4,ur0806494,tt7737800,7


In [6]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19250 entries, 0 to 19249
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               19250 non-null  object 
 1   img_url             19250 non-null  object 
 2   year                19250 non-null  int64  
 3   director            19242 non-null  object 
 4   stars               19173 non-null  object 
 5   duration            19141 non-null  object 
 6   genres              19232 non-null  object 
 7   overview            19220 non-null  object 
 8   rating              19250 non-null  float64
 9   num_rating          19250 non-null  object 
 10  num_user_review     19250 non-null  object 
 11  num_critic_review   19250 non-null  object 
 12  budget              9890 non-null   object 
 13  gross               13066 non-null  object 
 14  country             19221 non-null  object 
 15  metascore           11284 non-null  float64
 16  osca

In [7]:
df_movie['year'] = df_movie['year'].astype(str)

# Sort the DataFrame by 'year' in ascending order
df_movie = df_movie.sort_values(by='year')

In [8]:
null_counts = df_movie.isnull().sum()

print("Null counts in each column:")
print(null_counts)

Null counts in each column:
title                     0
img_url                   0
year                      0
director                  8
stars                    77
duration                109
genres                   18
overview                 30
rating                    0
num_rating                0
num_user_review           0
num_critic_review         0
budget                 9360
gross                  6184
country                  29
metascore              7966
oscar                 16488
win_and_nomination     5603
url                       0
dtype: int64


In [9]:
num_movies_before_deletion = len(df_movie)
print(f"Number of movies before deletion: {num_movies_before_deletion}")

important_columns = ['title', 'genres', 'rating', 'year', 'overview', 'duration']
df_movie = df_movie.dropna(subset=important_columns)

num_movies_after_deletion = len(df_movie)
print(f"Number of movies after deletion: {num_movies_after_deletion}")

Number of movies before deletion: 19250
Number of movies after deletion: 19108


In [10]:
len(df_movie.columns)

19

In [11]:
# Converting num_rating value, 'K' and 'M' to integers
def convert_to_int(value):
    if 'K' in value:
        return int(float(value.replace('K', '')) * 1000)
    elif 'M' in value:
        return int(float(value.replace('M', '')) * 1000000)
    else:
        return int(value)

df_movie['num_rating'] = df_movie['num_rating'].apply(convert_to_int)

In [12]:
unique_years = df_movie['year'].unique()
df_movie = df_movie[df_movie['year'].str.isnumeric()]
df_movie['year'] = df_movie['year'].astype(int)

In [13]:
# converting 'duration' to minutes
def convert_duration_to_minutes(duration_str):
    parts = duration_str.split()
    total_minutes = 0
    for part in parts:
        if 'h' in part:
            total_minutes += int(part.strip('h')) * 60
        elif 'm' in part:
            total_minutes += int(part.strip('m'))
    return total_minutes

# Apply the function to convert 'duration' values to minutes
df_movie['duration'] = df_movie['duration'].apply(convert_duration_to_minutes)

In [14]:
# Define a function to extract the number of Oscars won
def extract_oscar_count(oscar_str):
    if isinstance(oscar_str, str) and 'oscar' in oscar_str.lower() and 'won' in oscar_str.lower():
        words = oscar_str.split()
        for i in range(len(words)):
            if words[i].isdigit() and words[i + 1].lower() == 'oscar' and words[i - 1].lower() == 'won':
                return int(words[i])
    # If not 'won' and 'oscar' or no number found, return 0
    return 0

df_movie['oscar'] = df_movie['oscar'].apply(extract_oscar_count)

In [15]:
#budget processing
df_movie["budget"] = df_movie["budget"].astype(str)
df_movie["budget"] = df_movie["budget"].map(lambda x: x.rstrip(" (estimated)"))
df_movie["budget"] = df_movie["budget"].map(lambda x: x.replace(",", ""))

In [16]:
# split digit part and character part
df_movie["estimated_budget"] = df_movie["budget"].str.extract("(\d+)")
df_movie["budget_unit"] = df_movie["budget"].str.extract("(\D+)")

In [17]:
df_movie["budget_unit"].unique()
# if there exist a \xa0, replace it with a ' '
df_movie["budget_unit"] = df_movie["budget_unit"].map(lambda x: x.replace("\xa0", "").replace("\u202fCFA", ""))
df_movie["budget_unit"].unique()

array(['nan', '$', 'SEK', '¥', 'FRF', 'DEM', '£', 'ITL', 'PLN', 'RUR',
       'A$', 'NLG', 'CA$', 'TRL', '€', 'NZ$', 'NOK', 'IEP', 'DKK', 'HK$',
       'BEF', 'ESP', '₹', 'ATS', 'R$', 'SGD', 'FIM', 'HUF', 'CN¥', 'THB',
       '₩', 'CZK', 'CHF', '₪', 'ARS', 'MX$', 'MVR', 'IDR', 'LVL', 'NT$',
       '₱', 'MYR', 'PKR', 'BDT', 'NPR'], dtype=object)

In [18]:
currency_conversion = {
    '$': 1,    # Đô la Mỹ
    'A$': 0.75,  # Đô la Úc
    '£': 1.32,   # Bảng Anh
    'SEK': 0.11,  # Krona Thụy Điển
    'DKK': 0.16,  # Krone Đan Mạch
    '₹': 0.014,  # Rupee Ấn Độ
    '€': 1.18,   # Euro
    'CA$': 0.75,  # Đô la Canada
    'DEM': 0.61,  # Đức Mark
    '¥': 0.007,    # Yên Nhật
    'TRL': 0.13,  # Lira Thổ Nhĩ Kỳ
    'R$': 0.19,   # Real
    'CN¥': 0.15,  # Nhân dân tệ
    'PLN': 0.27,  # Zloty Ba Lan
    'FRF': 0.18,  # Pháp Franc
    '₩': 0.00084, # Won Hàn Quốc
    'FIM': 0.18,  # Markka Phần Lan
    'RUR': 0.013, # Ruble Nga
    'SGD': 0.74,  # Đô la Singapore
    'HUF': 0.0034,# Forint Hungary
    'NOK': 0.11,  # Krone Na Uy
    'MX$': 0.049, # Peso Mexico
    'NZ$': 0.69,  # Đô la New Zealand
    'HK$': 0.13,  # Đô la Hồng Kông
    'IDR': 0.000071, # Rupiah Indonesia
    'ESP': 0.0066, # Peseta Tây Ban Nha
    'ISK': 0.0079, # Króna Iceland
    'CHF': 1.09,  # Franc Thụy Sĩ
    'VEB': 0.0000000000000001, # Bolívar Venezuela
    'ALL': 0.0094, # Lek Albania
    'PKR': 0.0059, # Rupee Pakistan
    'BEF': 0.031, # Franc Bỉ
    'EGP': 0.064, # Bảng Ai Cập
    'DOP': 0.017, # Peso Dominica
    'ZAR': 0.067, # Rand Nam Phi
    'NGN': 0.0026, # Naira Nigeria
    'THB': 0.032, # Baht Thái Lan
    'IRR': 0.000024, # Rial Iran
    'LTL': 0.32, # Litas Litva
    'BDT': 0.012, # Taka Bangladesh
    'MYR': 0.24, # Ringgit Malaysia
    'LKR': 0.0054, # Rupee Sri Lanka
    'NT$': 0.035, # Đô la Đài Loan
    'CZK': 0.044, # Koruna Cộng hòa Séc
    'ARS': 0.011, # Peso Argentina
    'GEL': 0.31, # Lari Georgia
    'MNT': 0.00035, # Tugrik Mông Cổ
    'UZS': 0.000095, # Som Uzbekistan
    '₱': 0.021, # Peso Philipin
    'NLG': 0.59, # Guilder Hà Lan
    'ITL': 0.00061, # Lira Ý
    'NAD': 0.067, # Đô la Namibia
    'ROL': 0.00024, # Leu Rumani
    'EEK': 0.075, # Kroon Estonia
    'GTQ': 0.13, # Quetzal Guatemala
    'HNL': 0.041, # Lempira Honduras
    'XAU': 1.88, # Vàng
    'AED': 0.27, # Dirham UAE
    'COP': 0.00027, # Peso Colombia
    'PEN': 0.29, # Sol Peru
    'MOP': 0.12, # Pataca Ma Cao
    'SIT': 0.0059, # Tolar Slovenia
    'NPR': 0.0085, # Rupee Nepal
    'PTE': 0.0059, # Escudo Bồ Đào Nha
    '₪': 4.06,
    'BGL': 0.61, # Lev Bulgaria
    'MAD': 0.11, # Dirham Ma-rốc
    'UAH': 0.037, # Hryvnia Ukraina
    'MKD': 0.019, # Denar Macedonia
    'KES': 0.0092, # Shilling Kenya
    'HRK': 0.16, # Kuna Croatia
    'KWD': 3.29, # Dinar Kuwait
    'MMK': 0.00074, # Kyat Myanma
    'KZT': 0.0024, # Tenge Kazakhstan
    'AZM': 0.00012, # Manat Azerbaijan
    'GHC': 0.00018, # Cedi Ghana
    'JOD': 1.41, # Dinar Jordan
    '₫': 0.000043, # Đồng Việt Nam
    'ANG': 0.56, # Guilder Antille thuộc Hà Lan
    'LVL': 1.59, # Lats Latvia
    'MVR': 0.064, # Rufiyaa Maldives
    'LUF': 0.031, # Franc Luxembourg
    'UGX': 0.00027, # Shilling Uganda
    'IQD': 0.00084, # Dinar Iraq
    'GRD' : 0.0031, # Drachma Hy Lạp
    'LYD' : 0.23, # Dinar Libya
    'BYR' : 0.000041, # Ruble Belarus
    'AMD' : 0.0021, # Dram Armenia
    'TTD' : 0.15, # Đô la Trinidad và Tobago
    'RON' : 0.24, # Leu Romania
    'ATS' : 0.073, # Schilling Áo
    'BAM' : 0.61, # Mark Bosnia và Herzegovina
    'BSD' : 1, # Đô la Bahamas
    'OMR' : 2.60, # Rial Oman
    'CLP' : 0.0013, # Peso Chile
    'BHD' : 2.65, # Dinar Bahrain
    'SZL' : 0.067, # Lilangeni Swaziland
    'KGS' : 0.012, # Som Kyrgyzstan
    'ZMK' : 0.00011, # Kwacha Zambia
    'SLL' : 0.00010, # Leone Sierra Leone
    'TZS' : 0.00043, # Shilling Tanzania
    'DZD' : 0.0074, # Dinar Algeria
    'F' : 0.18, # Franc Pháp
    'TND' : 0.37, # Dinar Tunisia
    'CDF' : 0.00055, # Franc Congo
    'KHR' : 0.00025, # Riel Campuchia
    'AFA' : 0.014, # Afghani Afghanistan
    'BTN' : 0.013, # Ngultrum Bhutan
    'YUM' : 0.55, # Dinar Nam Tư
    'ETB' : 0.018, # Birr Ethiopia
    'CRC' : 0.0019, # Colón Costa Rica
    'TMM' : 0.29, # Manat Turkmenistan
    'QAR' : 0.27, # Rial Qatar
    'SYP' : 0.00008, # Bảng Syria
    'PYG' : 0.00015, # Guarani Paraguay
    'MUR' : 0.022, # Rupee Mauritius
    'FJD' : 0.45, # Đô la Fiji
    'IEP' : 1.40, # Bảng Ireland
    'SOS' : 0.0017, # Shilling Somalia
    'CUP' : 0.038, # Peso Cuba
    'SDD' : 0.0000166968, # Dinar Sudan
    'ZWD' : 0.0029, # Đô la Zimbabwe
    'EC$' : 0.37, # Đô la Đông Caribê
    'UYU' : 0.023, # Peso Uruguay
    'BOB' : 0.15, # Boliviano Bolivia
    'LBP' : 0.00066, # Bảng Li-băng
    'SKK' : 0.036, # Koruna Slovakia
    'RWF' : 0.0010, # Franc Rwanda
    'BIF' : 0.00052, # Franc Burundi
    'JMD' : 0.0067, # Đô la Jamaica
    'FCFA' : 0.0018, # Franc CFA Trung Phi
    'MTL' : 0.00028, # Lira Malta
    'KPW' : 0.0011, # Won Triều Tiên
    'BBD'  : 0.50, # Đô la Barbados
    'AWG' : 0.56, # Florin Aruba
}

def convert_budget_to_usd(row):
    if row['budget_unit'] in currency_conversion:
        # Convert the extracted budget to float and apply the exchange rate
        return float(row['estimated_budget']) * currency_conversion[row['budget_unit']]
    return float(row['estimated_budget'])

# Apply the conversion function to create a 'budget_usd' column
df_movie["budget_unit"].fillna('USD', inplace=True)  # Assuming missing units are USD
df_movie['budget_usd'] = df_movie.apply(convert_budget_to_usd, axis=1)

In [19]:
df_movie["gross"] = df_movie["gross"].astype(str)
df_movie["gross"] = df_movie["gross"].map(lambda x: x.replace(",", ""))
# split digit part and character part
df_movie["estimated_gross"] = df_movie["gross"].str.extract("(\d+)")
df_movie["gross_unit"] = df_movie["gross"].str.extract("(\D+)")

In [20]:
# get all the unique values of budget_unit
df_movie["gross_unit"].unique()
# if there exist a \xa0, replace it with a ' '
df_movie["gross_unit"] = df_movie["gross_unit"].map(lambda x: x.replace("\xa0", "").replace("\u202fCFA", ""))
df_movie["gross_unit"].unique()

array(['nan', '$', '₹'], dtype=object)

In [21]:
currency_conversion = {
    '$': 1,    # Đô la Mỹ
    '₹': 0.014,  # Rupee Ấn Độ
}

def convert_gross_to_usd(row):
    if row['gross_unit'] in currency_conversion:
        # Convert the extracted budget to float and apply the exchange rate
        return float(row['estimated_gross']) * currency_conversion[row['gross_unit']]
    return float(row['estimated_gross'])

# Apply the conversion function to create a 'budget_usd' column
df_movie["gross_unit"].fillna('USD', inplace=True)  # Assuming missing units are USD
df_movie['gross_usd'] = df_movie.apply(convert_gross_to_usd, axis=1)

In [22]:
# Extract wins and nominations and create new columns
wins = df_movie['win_and_nomination'].str.extract(r'(\d+) wins?')
nominations = df_movie['win_and_nomination'].str.extract(r'(\d+) nominations?')

# Fill missing values with 0
wins.fillna(0, inplace=True)
nominations.fillna(0, inplace=True)

# Add the extracted columns to the DataFrame
df_movie['win'] = wins
df_movie['nomination'] = nominations

In [23]:
# url -> movie_id
df_movie['movie_id'] = df_movie['url'].str.split('/').str[-2]

In [24]:
# drop column budget_unit, gross_unit, win_and_nomination
df_movie = df_movie.drop(['budget_unit', 'gross_unit', 'win_and_nomination'], axis=1)

In [25]:
# check if there is any duplicate movie_id
df_movie = df_movie.drop_duplicates(subset=['movie_id'])
df_movie['movie_id'].duplicated().sum()

0

In [26]:
df_movie.drop(['url', 'budget', 'gross', 'estimated_budget','estimated_gross'], axis=1, inplace=True)

In [27]:
# save the data
df_movie.to_csv('../data/cleaned/cleaned_movies_details.csv', index=False)

In [29]:
valid_movie_ids = set(df_rating['movie_id'].unique())
filtered_df_movie = df_movie[df_movie['movie_id'].isin(valid_movie_ids)]

filtered_df_rating = df_rating[df_rating['movie_id'].isin(filtered_df_movie['movie_id'])]

filtered_df_movie.to_csv('../data/cleaned/cleaned_movies_for_rating.csv', index=False)
filtered_df_rating.to_csv('../data/cleaned/cleaned_user_rating.csv', index=False)